In [26]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import csv
import random

In [27]:
def normalized(data):
    data_mean = np.mean(my_data, axis=0)
    data_std = np.std(my_data, axis=0)
    for i in range(len(data)):
        data[i] = (data[i] - data_mean) / data_std
    return data

In [28]:
def normalized2(data):
    data_max = np.max(my_data, axis=0)
    data_min = np.min(my_data, axis=0)
#     print(data_max)
#     print(data_min)
#     print(data_max-data_min)
    for i in range(len(data)):
        data[i] = (data[i] - data_min) / (data_max - data_min)
    return data

In [29]:
def data_append(n_data, data):
    data_ex = np.zeros((len(data), 2), dtype = 'float')
    for i in range(len(data)):
        data_ex[i][0] = data[i][1] / data[i][0]
        data_ex[i][1] = data[i][3] / data[i][2]
    data = np.concatenate((n_data, data_ex),axis=1)
    return data

In [30]:
def normalized4(data):
    data_min = np.min(my_data, axis=0)
    data_max = np.max(my_data, axis=0)
    n_data = np.zeros(data.shape, dtype = 'int')
    for i in range(len(data)):
        for j in range(len(data[i])):
            if j <4:
                n_data[i][j] = (data[i][j] - data_min[j])*10
            else:
                n_data[i][j] = (data[i][j] - data_min[j])*100
#     print(data_max)
#     print(data_min)
#     print(data_max-data_min)
    return n_data

In [31]:
def shuffle(data_full):
    random.shuffle(data_full)
    data = []
    data_class = []
#     print(data_full[0:5])
    for i in range(len(data_full)):
        data.append(data_full[i][0:4])
        data_class.append(data_full[i][4])
#     print(data[0:5])
#     print(data_class[0:5])
    return data, data_class

In [43]:
data = []
data_class = []
data_full = []
with open('./iris.csv') as f:
    myCsv = csv.reader(f)
    headers = next(myCsv)
    for row in myCsv:
        data_full.append(row)

data_count = len(data)
data, data_class = shuffle(data_full)
my_data = np.array(data, dtype = 'float')
print(my_data[0])
n_data = normalized4(my_data)
print(n_data[0])

[7.2 3.  5.8 1.6]
[29 10 48 15]


In [33]:
W = []
M = 150
N = 24
nc = 3

In [34]:
def get_weights():
    y = np.random.random() * (2.0 / np.sqrt(M))
    return 0.1 - (1 / np.sqrt(M)) + y

In [35]:
for i in range(nc):
    W.append(list())
    for j in range(N):
        W[i].append(get_weights()*0.5)
print(W)
wwe = np.array(W, dtype='float')
print(wwe)
wwe = wwe.T
print(wwe)

[[0.05825471104257193, 0.04392843603533553, 0.08874040506473155, 0.020041332793213566, 0.07073236426322224, 0.08347894785026089, 0.019665958308310076, 0.031780755587518486, 0.053224606580861256, 0.038183998514026415, 0.013956799007366218, 0.03974951336615873, 0.0813504979007306, 0.019940764802533935, 0.07411137931221035, 0.04309913064381354, 0.06551048784656326, 0.05892782824527514, 0.055583063107854554, 0.010607971639133278, 0.05792214941297596, 0.04365509310532373, 0.0826763267724673, 0.010789071246799547], [0.04959451568753484, 0.05659032522573807, 0.05737849159009811, 0.054318732386020925, 0.05293102564798401, 0.0781249688146515, 0.07907652905802019, 0.049368775902753535, 0.07872091968566228, 0.01081105273193728, 0.0786271161615992, 0.010209802262471954, 0.08758805954239056, 0.03301308364561536, 0.057212859327640836, 0.04394578240229014, 0.04885961376091795, 0.0890229695611915, 0.01978411430849439, 0.07908797122654901, 0.019790824692194404, 0.07327983682353192, 0.0585063724726583, 

In [36]:
def initWeight(input_dim, output_dim):
    return tf.Variable(tf.nn.softmax(tf.random_normal([input_dim, output_dim]), axis=0), name='Weight')

In [37]:
def forward(input_dim, output_dim, inputs, Weight):
    outputs = tf.matmul(inputs, Weight)
    return outputs  

In [38]:
def update(inputs, outputs, Weight, g):
    tmp = 0
    ans = 0
    for i in range(len(outputs[0])):
        if outputs[0][i] > tmp:
            ans = i
            tmp = outputs[0][i]        
    M = np.sum(inputs[0])
    for i in range(len(inputs[0])):
        Weight[i][ans] += ((inputs[0][i]/M) - Weight[i][ans])*g
    return Weight

In [39]:
def predict(inputs, Weight):
    (layer_X, layer_y) = sess.run((X, y), feed_dict={X:inputs, _W:Weight})
    tmp = 0
    ans = 0
    for i in range(len(layer_y[0])):
        if layer_y[0][i] > tmp:
            ans = i
            tmp = layer_y[0][i]  
#     print(ans)
    return ans

In [40]:
def encode(X_array):
    en_array = np.zeros(24)
    for i in range(len(X_array[0])):
        x = X_array[0][i]
        x_trans = np.zeros(6)
        key = 0
        while(x != 0):
            x_trans[key] = x % 2
            x = int(x /2)
            key += 1
        en_array[i*6:(i+1)*6] = x_trans
    en_array = en_array.reshape((1, 24))
    return en_array

In [42]:
epochs = 200
k = 5  # use k-flod
num_val = len(n_data)//k
g = 0.6
input_num = 24
output_num = 3
# ans_count = np.zeros((3, 3), dtype=np.int)

X = tf.placeholder("float", [None, input_num])
_W = initWeight(input_dim = input_num, output_dim = output_num)
y = forward(input_dim = input_num, output_dim = output_num, inputs = X, Weight = _W)


with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    Weights = sess.run(_W)
#     Weights = wwe
#     print(Weights)
    for n in range(k):
        print('Processing fold #', n)
        val_data = n_data[n * num_val: (n + 1) * num_val]
        val_class = data_class[n * num_val: (n + 1) * num_val]
        train_data = np.concatenate([n_data[:n * num_val], n_data[(n + 1) * num_val:]], axis=0)
        for epoch in range(epochs):
#             if epoch==epochs/2:
#                 g = 0.7
            for i in range(len(train_data)):
                key = random.randint(0, len(train_data)-1)
                X_array = np.array([train_data[key]])
                X_array = encode(X_array)
                (layer_X, layer_y) = sess.run((X, y), feed_dict={X:X_array, _W:Weights})
                Weights = update(layer_X, layer_y, Weights, g)
            print("epoch: {:d} / {:d}".format(epoch+1, epochs), end="\r", flush=True)
#     print("\n",Weights)
        ans_count = np.zeros((3, 3), dtype=np.int)
        for i in range(len(val_data)):
            X_array = np.array([val_data[i]])
            X_array = encode(X_array)
            ans = predict(X_array, Weights)
            if val_class[i] == 'Setosa':
                ans_count[0][ans] += 1
            if val_class[i] == 'Versicolor':
                ans_count[1][ans] += 1
            if val_class[i] == 'Virginica':
                ans_count[2][ans] += 1
        print("\n",ans_count)
        g -= 0.1
    ans_count = np.zeros((3, 3), dtype=np.int)
    for i in range(len(n_data)):
        X_array = np.array([n_data[i]])
        X_array = encode(X_array)
        ans = predict(X_array, Weights)
        if data_class[i] == 'Setosa':
            ans_count[0][ans] += 1
        if data_class[i] == 'Versicolor':
            ans_count[1][ans] += 1
        if data_class[i] == 'Virginica':
            ans_count[2][ans] += 1
    print("\n",ans_count)     

Processing fold # 0
epoch: 200 / 200
 [[1 1 6]
 [7 2 2]
 [3 4 4]]
Processing fold # 1
epoch: 200 / 200
 [[10  0  0]
 [ 5  1  1]
 [ 6  1  6]]
Processing fold # 2
epoch: 200 / 200
 [[ 5  0  4]
 [ 2 11  2]
 [ 4  2  0]]
Processing fold # 3
epoch: 200 / 200
 [[ 0 11  0]
 [ 0  3  8]
 [ 6  2  0]]
Processing fold # 4
epoch: 200 / 200
 [[ 0 12  0]
 [ 0  0  6]
 [ 8  2  2]]

 [[ 0 50  0]
 [ 4  6 40]
 [36  8  6]]


In [32]:
def encode(X_array):
    en_array = np.zeros( 24)
    for i in range(len(X_array[0])):
        x = X_array[0][i]
        x_trans = np.zeros(6)
        key = 0
        while(x != 0):
            x_trans[key] = x % 2
            x = int(x /2)
            key += 1
        en_array[i*6:(i+1)*6] = x_trans
    en_array = en_array.reshape((1, 24))
    return en_array

In [33]:
X_array = np.array([n_data[0]])
X_array = encode(X_array)

In [82]:
num_val = len(n_data)//k
print(num_val)
train_data = np.concatenate([n_data[:n * num_val], n_data[(i + 1) * num_val:]], axis=0)
print(len(train_data))
key = random.randint(0, len(train_data)-1)

30
120


In [ ]:
print()